In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input, decode_predictions

# Cargar el modelo
modelo = tf.keras.models.load_model('models/optimizado.keras')

# Cargar y preprocesar la imagen
img_path = 'ImagenPrueba.jpg'
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

# Hacer una predicción
preds = modelo.predict(x)
print('Predicción:', decode_predictions(preds, top=1)[0])

# Usar Grad-CAM para generar el mapa de calor
grad_model = tf.keras.models.Model([modelo.inputs], [modelo.get_layer('block5_conv3').output, modelo.output])
with tf.GradientTape() as tape:
    conv_outputs, predictions = grad_model(x)
    loss = predictions[:, np.argmax(predictions[0])]

grads = tape.gradient(loss, conv_outputs)[0]
pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
conv_outputs = conv_outputs[0]

for i in range(pooled_grads.shape[-1]):
    conv_outputs[:, :, i] *= pooled_grads[i]

heatmap = np.mean(conv_outputs, axis=-1)
heatmap = np.maximum(heatmap, 0)
heatmap /= np.max(heatmap)

# Mostrar el mapa de calor
plt.matshow(heatmap)
plt.show()


ValueError: File not found: filepath=models/optimizado.keras. Please ensure the file is an accessible `.keras` zip file.